In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib
import sys

import matplotlib
import numpy as np

matplotlib.rcParams["image.interpolation"] = "none"
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob

import tifffile
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.utils import Path, normalize
from stardist import random_label_cmap
from stardist.models import StarDist3D
from tifffile import imread

np.random.seed(6)
lbl_cmap = random_label_cmap()

2024-12-05 12:48:17.951500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733428097.984568  725133 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733428097.993662  725133 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 12:48:18.071129: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# set path to the test image
image_file_path = pathlib.Path("../../data/z-stack_images/D3-1/D3-1_405.tif").resolve(
    strict=True
)

nuclei_z_stack = tifffile.imread(image_file_path)
print(nuclei_z_stack.shape)

(23, 1537, 1540)


## Onmipose

In [3]:
# Import dependencies
import numpy as np
from cellpose_omni import core, models

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you
# are only processing a few images.
use_GPU = core.use_gpu()
print(">>> GPU activated? %d" % use_GPU)

# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams["figure.dpi"] = 300
plt.style.use("dark_background")
%matplotlib inline

2024-12-05 12:48:32,824	[INFO ]	[core.py    74   _use_gpu_torch    ]	** TORCH GPU version installed and working. **
>>> GPU activated? 1


In [4]:
import os
import pathlib

import omnipose
from cellpose_omni import io

basedir = pathlib.Path("../test_dir/").resolve(strict=True)
files = io.get_image_files(str(basedir))
files  # this displays the variable if it the last thing in the code block

['/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/1.process_images/test_dir/D3-1_405.tif']

In [5]:
from cellpose_omni import io, transforms
from omnipose.utils import normalize99

imgs = [io.imread(f) for f in files]

# print some info about the images.
for i in imgs:
    print("Original image shape:", i.shape)
    print("data type:", i.dtype)
    print("data range:", i.min(), i.max())
nimg = len(imgs)
print("number of images:", nimg)

Original image shape: (23, 1537, 1540)
data type: uint16
data range: 0 65535
number of images: 1


In [6]:
from cellpose_omni import models

model_name = "plant_omni"

dim = 3
nclasses = 3  # flow + dist + boundary
nchan = 1
omni = 1
rescale = False
diam_mean = 0
use_GPU = 0  # Most people do not have enough VRAM to run on GPU... 24GB not enough for this image, need nearly 48GB
model = models.CellposeModel(
    gpu=use_GPU,
    model_type=model_name,
    net_avg=False,
    diam_mean=diam_mean,
    nclasses=nclasses,
    dim=dim,
    nchan=nchan,
)

2024-12-05 12:48:39,033	[INFO ]	[models.py  427  __init__          ]	>>plant_omni<< model set to be used
2024-12-05 12:48:39,034	[INFO ]	[core.py    88   assign_device     ]	>>>> using CPU


resnet_torch.py (308): You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


In [10]:
from cellpose_omni import models

model_name = "nuclei"

dim = 3
nclasses = 3  # flow + dist + boundary
nchan = 1
omni = 1
rescale = False
diam_mean = 0
use_GPU = True  # Most people do not have enough VRAM to run on GPU... 24GB not enough for this image, need nearly 48GB
model = models.CellposeModel(
    gpu=use_GPU,
    model_type=model_name,
    net_avg=False,
    diam_mean=diam_mean,
    nclasses=nclasses,
    dim=dim,
    nchan=nchan,
)

2024-12-05 12:50:57,385	[INFO ]	[models.py  427  __init__          ]	>>nuclei<< model set to be used
2024-12-05 12:50:57,399	[INFO ]	[core.py    74   _use_gpu_torch    ]	** TORCH GPU version installed and working. **
2024-12-05 12:50:57,400	[INFO ]	[core.py    85   assign_device     ]	>>>> using GPU


RuntimeError: Error(s) in loading state_dict for CPnet:
	size mismatch for downsample.down.res_down_0.conv.conv_0.2.weight: copying a param with shape torch.Size([32, 2, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 2, 3, 3, 3]).
	size mismatch for downsample.down.res_down_0.conv.conv_1.2.weight: copying a param with shape torch.Size([32, 32, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 32, 3, 3, 3]).
	size mismatch for downsample.down.res_down_0.conv.conv_2.2.weight: copying a param with shape torch.Size([32, 32, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 32, 3, 3, 3]).
	size mismatch for downsample.down.res_down_0.conv.conv_3.2.weight: copying a param with shape torch.Size([32, 32, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 32, 3, 3, 3]).
	size mismatch for downsample.down.res_down_0.proj.1.weight: copying a param with shape torch.Size([32, 2, 1, 1]) from checkpoint, the shape in cu

In [8]:
import torch

torch.cuda.empty_cache()
mask_threshold = -5  # usually this is -1
flow_threshold = 0.0
diam_threshold = 12
net_avg = False
cluster = False
verbose = 1
tile = 0
chans = None
compute_masks = 1
resample = False
rescale = None
omni = True
flow_factor = 10  # multiple to increase flow magnitude, useful in 3D
transparency = True

nimg = len(imgs)
masks_om, flows_om = [[]] * nimg, [[]] * nimg

# splitting the images into batches helps manage VRAM use so that memory can get properly released
# here we have just one image, but most people will have several to process
for k in range(nimg):
    masks_om[k], flows_om[k], _ = model.eval(
        imgs[k],
        channels=chans,
        rescale=rescale,
        mask_threshold=mask_threshold,
        net_avg=net_avg,
        transparency=transparency,
        flow_threshold=flow_threshold,
        omni=omni,
        resample=resample,
        verbose=verbose,
        diam_threshold=diam_threshold,
        cluster=cluster,
        niter=10,
        tile=tile,
        compute_masks=compute_masks,
        flow_factor=flow_factor,
    )

2024-12-05 10:39:19,759	[WARNING]	[models.py  699  eval              ]	input images must be a list of images, array of images, or dataloader
2024-12-05 10:39:19,760	[INFO ]	[models.py  709  eval              ]	Evaluating with flow_threshold 0.00, mask_threshold -5.00
2024-12-05 10:39:19,760	[INFO ]	[models.py  711  eval              ]	using omni model, cluster False
2024-12-05 10:39:19,760	[INFO ]	[models.py  1097 eval              ]	not using dataparallel
failed to load model Error(s) in loading state_dict for CPnet:
	size mismatch for downsample.down.res_down_0.conv.conv_0.2.weight: copying a param with shape torch.Size([32, 2, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 2, 3, 3, 3]).
	size mismatch for downsample.down.res_down_0.conv.conv_1.2.weight: copying a param with shape torch.Size([32, 32, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 32, 3, 3, 3]).
	size mismatch for downsample.down.res_down_0.conv.conv_2.2.weight: copying a 

In [ ]:
%%capture
import ncolor

mask = masks_om[0]
mask_nc = ncolor.label(mask, max_depth=20)

import napari

viewer = napari.view_labels(mask_nc)
viewer.dims.ndisplay = 3
viewer.camera.center = [s // 2 for s in mask.shape]
viewer.camera.zoom = 1
viewer.camera.angles = (10.90517458968619, -20.777067798396835, 58.04311170773853)
viewer.camera.perspective = 0.0
viewer.camera.interactive = True

img = viewer.screenshot(size=(1000, 1000), scale=1, canvas_only=True, flash=False)

In [ ]:
plt.figure(figsize=(3, 3), frameon=False)
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
from cellpose_omni import plot
from omnipose.plot import apply_ncolor

mu = flows_om[0][1]
T = flows_om[0][2]
bd = flows_om[0][4]
# mu.shape,T.shape,bd.shape

d = mu.shape[0]

from omnipose.utils import rescale

c = np.array([1] * 2 + [0] * (d - 2))
# c = np.arange(d)


def cyclic_perm(a):
    n = len(a)
    b = [[a[i - j] for i in range(n)] for j in range(n)]
    return b


slices = []
idx = np.arange(d)
cmap = mpl.colormaps["magma"]
cmap2 = mpl.colormaps["viridis"]

for inds in cyclic_perm(c):
    slc = tuple([slice(-1) if i else mu.shape[k + 1] // 2 for i, k in zip(inds, idx)])
    flow = plot.dx_to_circ(mu[np.where(inds) + slc], transparency=1) / 255
    dist = cmap(rescale(T)[slc])
    bnds = cmap2(rescale(bd)[slc])
    msks = apply_ncolor(masks_om[0][slc])

    fig = plt.figure(figsize=[5] * 2, frameon=False)
    plt.imshow(np.hstack((flow, dist, bnds, msks)), interpolation="none")
    plt.axis("off")
    plt.show()

In [ ]:
from cellpose_omni import core, models

# define cellpose model
model_name = "plant_cp"

# this model was trained on 2D slices
dim = 2
nclasses = 2  # cellpose models have no boundary field, just flow and distance

# Cellpose defaults to 2 channels;
# this is the setup for grayscale in that case
nchan = 2
chans = [0, 0]

# no rescaling for this model
diam_mean = 0


use_GPU = core.use_gpu()
model = models.CellposeModel(
    gpu=use_GPU,
    model_type=model_name,
    net_avg=False,
    diam_mean=diam_mean,
    nclasses=nclasses,
    dim=dim,
    nchan=nchan,
)


# segmentation parameters
omni = 1
rescale = False
mask_threshold = 0
net_avg = 0
verbose = 1
tile = 0
compute_masks = 1
rescale = None
flow_threshold = 0.0
do_3D = True
flow_factor = 10

masks_cp, flows_cp, _ = model.eval(
    imgs,
    channels=chans,
    rescale=rescale,
    mask_threshold=mask_threshold,
    net_avg=net_avg,
    transparency=True,
    flow_threshold=flow_threshold,
    verbose=verbose,
    tile=tile,
    compute_masks=compute_masks,
    do_3D=True,
    omni=omni,
    flow_factor=flow_factor,
)

In [ ]:
%%capture
import ncolor

mask = masks_cp[0]
mask_nc = ncolor.label(mask, max_depth=20)

import napari

viewer = napari.view_labels(mask_nc)
viewer.dims.ndisplay = 3
viewer.camera.center = [s // 2 for s in mask.shape]
viewer.camera.zoom = 1
viewer.camera.angles = (10.90517458968619, -20.777067798396835, 58.04311170773853)
viewer.camera.perspective = 0.0
viewer.camera.interactive = True

img = viewer.screenshot(size=(1000, 1000), scale=1, canvas_only=True, flash=False)
plt.figure(figsize=(3, 3), frameon=False)
plt.imshow(img)
plt.axis("off")
plt.show()

## Cellpose

In [ ]:
import tensorflow as tf
from cellpose_omni import core, models
from skimage import io
from skimage.exposure import rescale_intensity

# Clear any previous sessions
tf.keras.backend.clear_session()

# Load your image data
nuclei_z_stack

# Check the shape of the array
print(nuclei_z_stack.shape)

# Normalize the image
# Adjust the axis based on the shape of nuclei_z_stack
# axis_norm = 0  # or 1, depending on the shape
# img = rescale_intensity(nuclei_z_stack, in_range=(1, 99.8), out_range=(0, 1))
img = nuclei_z_stack
# Initialize the model
model_name = "nuclei"
use_GPU = core.use_gpu()
model = models.CellposeModel(gpu=use_GPU, model_type=model_name)

# Perform segmentation
labels, details, _ = model.eval(img, diameter=50, channels=[0, 0])

In [ ]:
# plot the image and masks
fig = plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(nuclei_z_stack[8, :, :], cmap="gray")
plt.title("image")
plt.axis("off")
plt.subplot(122)
plt.imshow(labels[8], cmap="magma")
plt.title("masks")
plt.axis("off")
plt.show()

In [ ]:
# show each z slice of the image and masks
for z in range(nuclei_z_stack.shape[0]):
    fig = plt.figure(figsize=(10, 5))
    plt.subplot(121)
    plt.imshow(nuclei_z_stack[z, :, :], cmap="gray")
    plt.title("image")
    plt.axis("off")
    plt.subplot(122)
    plt.imshow(labels[z], cmap="magma")
    plt.title("masks")
    plt.axis("off")
    plt.show()